This notebook builds a smart finance assistant chatbot that leverages AI to help users track and manage their personal transactions. The chatbot extracts financial details from user input using a Language Model (LLM), categorizes income and expenses, and logs them into a database. **It personalizes categories for each user**, validates entries, and provides insightful feedback about spending habits. By combining natural language understanding with real-time database integration, this chatbot offers a seamless and intelligent way to automate personal finance management and budgeting.

## 1- Import Libraries 

In [1]:
import os
from langchain_fireworks import Fireworks
from langchain.memory import ConversationBufferMemory
from datetime import datetime
from langchain.retrievers import EnsembleRetriever
import re
from dateparser import parse as date_parse
from datetime import datetime
from supabase import create_client, Client
from dotenv import load_dotenv

## 2- Set API key 

In [ ]:
# Load the .env file
load_dotenv()

# Access the variables
api_key = os.getenv("API_KEY")

## 3- Call LLM 

In [3]:
llm = Fireworks(api_key=api_key, model="accounts/fireworks/models/deepseek-v3", temperature=1.0, max_tokens=1024)
response = llm.invoke("Hello, how are you?")
print(response)

 I'm Andrea, and I teach Italian as a foreign language. I'm a native speaker of Italian, and I currently live in Rome, Italy.

I hold a Bachelor's Degree in Languages and Intercultural Communication and a Master's Degree in Foreign Languages for International Communication, both from the University of Naples "L'Orientale." I also have a DITALS certification (level I) from the University for Foreigners of Siena.

After my studies, I worked in the tourism sector as a tour guide in archaeological sites in Southern Italy. I also collaborated with institutions that provided linguistic assistance to asylum seekers and migrants. Currently, I am involved in teaching Italian as a second language to a class of foreign students in lower secondary school.

Thanks to this diverse experience, I have interacted with people from all over the world, from teenagers and young adults to mature individuals. However, I realized that my greatest passion lies in teaching Italian as a foreign language in a mor

## 4- Create memory

In [4]:
#  Initialize memory
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)


C:\Users\Tony\AppData\Local\Temp\ipykernel_15460\1343280667.py:2: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)


## 5- Fetch database

In [ ]:
import os
from dotenv import load_dotenv
from supabase import create_client, Client

# Load .env file
load_dotenv()

# Get credentials from environment
url = os.getenv("SUPABASE_URL")
key = os.getenv("SUPABASE_KEY")

# Connect to Supabase
supabase: Client = create_client(url, key)

In [6]:
# Fetch data from the "Transaction" table
response = supabase.table("transactions").select("*").execute()
data = response.data
print(data)

[{'transaction_id': '3683c4fc-018d-42ee-803b-69845dd1f0cd', 'user_id': '027051a8-3887-4150-9cfb-a51efb9146b5', 'income': 7000, 'expenses': 0, 'data': 'Monthly salary deposit', 'category_id': '8867173b-22a3-408e-a2b8-9ee2f0bc70b2', 'description': 'August salary', 'created_at': '2025-04-21T10:51:09.833846'}, {'transaction_id': '2f662481-3b4e-47e3-aeec-f0ca06c7c7fb', 'user_id': 'eb242352-2899-4717-955c-7247db8a40ef', 'income': 0, 'expenses': 150.75, 'data': None, 'category_id': '813c1667-760d-46c2-8226-dc139cc8c86c', 'description': 'Bought groceries from Carrefour', 'created_at': '2025-05-07T09:50:25.344434'}, {'transaction_id': 'bd7bd415-064b-4d9c-a0c3-4942e5fd1a43', 'user_id': 'eb242352-2899-4717-955c-7247db8a40ef', 'income': 0, 'expenses': 250, 'data': None, 'category_id': '114ed927-ca6b-47df-bf3d-689ff3b8bbd9', 'description': 'Weekly groceries at Carrefour', 'created_at': '2025-05-07T09:53:27.097738'}, {'transaction_id': 'f7441b54-0fe6-4772-ad92-dcf1c64c44cd', 'user_id': 'eb242352-289

In [7]:
# Fetch data from the "categories" table
response = supabase.table("categories").select("*").execute()
data = response.data
print(data)

[{'category_id': '337bf3ed-8f2e-4647-ac50-30f327316d07', 'category_type': 'Food', 'category_name': 'Caffe', 'user_id': None}, {'category_id': 'c45b8dff-e08f-4c36-b30c-68ae7026b2f8', 'category_type': 'Transportation', 'category_name': 'taxi', 'user_id': None}, {'category_id': '3782b789-59fa-45b1-a214-c29e21a2f0e3', 'category_type': 'transportation', 'category_name': 'fuel', 'user_id': None}, {'category_id': '8867173b-22a3-408e-a2b8-9ee2f0bc70b2', 'category_type': 'Expense', 'category_name': 'Groceries', 'user_id': '027051a8-3887-4150-9cfb-a51efb9146b5'}, {'category_id': 'a8a017fc-e86f-4c23-9455-4f8db846c59b', 'category_type': 'Income', 'category_name': 'Salary', 'user_id': '47b66d92-1ea1-4344-af56-0771ed917456'}, {'category_id': 'af91ae83-0d65-4bd1-880f-747f3ebbaf35', 'category_type': 'Expense', 'category_name': 'Utilities', 'user_id': '99eed9c3-78dc-4925-b599-b02332524f78'}, {'category_id': '813c1667-760d-46c2-8226-dc139cc8c86c', 'category_type': 'expense', 'category_name': 'Groceries'

## 6- Fetch user categories

In [17]:
def fetch_user_categories(user_id, supabase, type_group="expense"):
    # type_group should be "expense" or "income" (case-insensitive)
    cat_type = 'Expense' if type_group.lower() == "expense" else 'Income'
    # Fetch only categories belonging to this user and of the correct type
    response = (
        supabase
        .table("categories")
        .select("*")
        .eq("user_id", user_id)
        .eq("category_type", cat_type)
        .execute()
    )
    cat_data = response.data
    # Return just the names
    categories = [c["category_name"] for c in cat_data]
    return categories

## 7- Prompt

In [12]:
def process_transaction_with_llm(text, user_id, supabase):
    # Get dynamic categories for this user
    expense_categories = fetch_user_categories(user_id, supabase, "expense")
    income_categories = fetch_user_categories(user_id, supabase, "income")

    # Join for prompt
    expense_list = ", ".join(expense_categories)
    income_list = ", ".join(income_categories)
    # Prompt for extracting transaction details
    transaction_prompt = f"""
    Extract transaction details from the following text and classify the amount as either income or expense.
    If it's an expense, put amount in EXPENSE; if it’s income, put amount in INCOME. Extract date (handle relative dates like 'yesterday'), a description, and the category.
    If a specific date is not mentioned, assume today's date. If you cant classify the amount as either income or expense,
    assume it is expense and keep it uncategorized. Provide a natural response about the spending.

    Text: {{input_text}}

    Current date: {{current_date}}

    Format your response as follows:
    DATE: [Extract the date, handle relative dates like 'this morning', 'yesterday', 'last week', etc.]
    EXPENSE: [amount, if spending; leave blank if not]
    INCOME: [amount, if income; leave blank if not]
    CATEGORY: [Classify into one of if expense: {expense_list}
               Classify into one of if income: {income_list}]
    Description: [one or two word description of the transaction]
    FEEDBACK: [Provide a natural, conversational response about the spending. Consider:
    - If it's a good deal or expensive for that category
    - Suggest money-saving tips if relevant
    - Compliment good financial decisions
    - Express concern for unusually high spending
    - Comment on the timing or necessity of the purchase
    Make it sound natural and varied.]

    Example:
    Input: "I got today 2,000 EGP from upwork"
    DATE: 2024-01-30
    INCOME: 2000
    CATEGORY: Part-time
    DESCRIPTION: upwork
    FEEDBACK: That’s wonderful—congrats on your part-time job!
    
    Input: "Bought lunch for 50 EGP yesterday"
    DATE: 2024-01-29
    EXPENSE: 50
    CATEGORY: Food
    DESCRIPTION: lunch
    FEEDBACK: That's a reasonable amount for lunch! If you're looking to save more, you might consider bringing lunch from home occasionally.
    """

    # Prepare the prompt with current date
    current_date = datetime.now().strftime('%Y-%m-%d')
    formatted_prompt = transaction_prompt.format(
        input_text=text,
        current_date=current_date
    )

    # Get response from LLM
    response = llm.invoke(formatted_prompt)
    
    # Parse the response (you might need to adjust this based on your LLM's output format)
    return response



this user has no food category so if dinner categorized as Groceries the test is successed

In [19]:
print(process_transaction_with_llm("Had dinner for 150 EGP last night", user_id="027051a8-3887-4150-9cfb-a51efb9146b5", supabase=supabase))


 

    DATE: 2025-05-09
    EXPENSE: 150
    CATEGORY: Groceries
    DESCRIPTION: dinner
    FEEDBACK: Dinner out can be a nice treat, but it’s worth keeping an eye on how often you dine out to manage your budget better.


In [22]:
print(process_transaction_with_llm("work on tasks got 500", user_id="027051a8-3887-4150-9cfb-a51efb9146b5", supabase=supabase))

 
    DATE: 2025-05-10
    INCOME: 500
    CATEGORY: Freelance
    DESCRIPTION: tasks
    FEEDBACK: Great job! Keep up the good work and continue growing your earnings.


In [23]:
print(process_transaction_with_llm("got 500", user_id="027051a8-3887-4150-9cfb-a51efb9146b5", supabase=supabase))




Input: "got 500"
DATE: 2025-05-10
INCOME: 500
CATEGORY: Salary
DESCRIPTION: cash
FEEDBACK: Getting cash is always a great boost! Make sure to plan how you’ll use it wisely.


In [24]:
print(process_transaction_with_llm("SON 200", user_id="027051a8-3887-4150-9cfb-a51efb9146b5", supabase=supabase))

 

    DATE: 2025-05-10
    EXPENSE: 200
    CATEGORY: Kids
    DESCRIPTION: SON
    FEEDBACK: Spending on your son is always meaningful. I hope it brought him joy or helped with something important!


In [27]:
print(process_transaction_with_llm("catii eat 200", user_id="027051a8-3887-4150-9cfb-a51efb9146b5", supabase=supabase))

 

    DATE: 2025-05-10
    EXPENSE: 200
    CATEGORY: Pets
    DESCRIPTION: catii
    FEEDBACK: Taking care of your pet is important, but keep an eye on expenses to ensure you’re staying within your budget!


## 8- Parse response

In [28]:
import re
from datetime import datetime
from dateparser import parse as date_parse

def parse_llm_response(response):
    # Extract date (default to today if not found or "Unknown")
    date_match = re.search(r"DATE:\s*(.*)", response)
    date_str = date_match.group(1).strip() if date_match else None
    if not date_str or date_str.lower() == "unknown":
        date_obj = datetime.now()
    else:
        date_obj = date_parse(date_str)
        if not date_obj:
            date_obj = datetime.now()
    date_iso = date_obj.strftime("%Y-%m-%d")

    # Extract EXPENSE and INCOME
    expense_match = re.search(r"EXPENSE:\s*([\d.]+)", response, re.IGNORECASE)
    income_match = re.search(r"INCOME:\s*([\d.]+)", response, re.IGNORECASE)
    expinc_match = re.search(r"EXPENSE/INCOME:\s*([\d.]+)", response, re.IGNORECASE)

    if expense_match:
        expense = float(expense_match.group(1))
        income = 0.0
    elif income_match:
        expense = 0.0
        income = float(income_match.group(1))
    elif expinc_match:
        expense = float(expinc_match.group(1))
        income = 0.0  #  default this to expense!
    else:
        expense = 0.0
        income = 0.0

    # Extract DESCRIPTION
    desc_match = re.search(r"DESCRIPTION:\s*(.*)", response, re.IGNORECASE)
    description = desc_match.group(1).strip() if desc_match and desc_match.group(1).strip() else None

    # Extract CATEGORY (optional)
    category_match = re.search(r"CATEGORY:\s*(.*)", response)
    category = category_match.group(1).strip() if category_match and category_match.group(1).strip() else None

    # Extract FEEDBACK (optional)
    feedback_match = re.search(r"FEEDBACK:\s*(.*)", response)
    feedback = feedback_match.group(1).strip() if feedback_match else ""

    return {
        "date": date_iso,
        "expense": expense,
        "income": income,
        "category": category,
        "description": description,
        "feedback": feedback
    }



## 9- Get category_id

In [29]:
def get_category_id(category_name: str, supabase):
    # Returns None if category_name is None or not found
    if not category_name:
        return None
    cat_response = supabase.table("categories").select("category_id").eq("category_name", category_name).execute()
    cat_data = cat_response.data
    if cat_data:
        return cat_data[0]["category_id"]
    else:
        return None


## 10- Add transaction into DB

In [30]:
def add_transaction_from_llm(llm_response, user_id, supabase):
    extracted = parse_llm_response(llm_response)
    description_valid = extracted["description"] not in (None, "", "No Description")
    amount_valid = (extracted["expense"] > 0 or extracted["income"] > 0)

    if not description_valid or not amount_valid:
        return "❌ Cannot add transaction: missing information."

    category_id = get_category_id(extracted["category"], supabase)
    new_transaction = {
        "income": extracted["income"],
        "expenses": extracted["expense"],
        "description": extracted["description"],
        "created_at": extracted["date"],
        "user_id": user_id,
        "category_id": category_id
    }
    response = supabase.table("transactions").insert(new_transaction).execute()

    # Build friendly message for user
    # Compose a string stating what was added, and echo the LLM's feedback
    tx_type = "income" if extracted["income"] > 0 else "expense"
    tx_value = extracted["income"] if extracted["income"] > 0 else extracted["expense"]

    user_reply = (
        f"✅ Added {tx_type} transaction: {extracted['description']}, "
        f"{tx_value} EGP, "
        f"date: {extracted['date']}"
    )
    if extracted["category"]:
        user_reply += f", category: {extracted['category']}."
    else:
        user_reply += "."
    if extracted["feedback"]:
        user_reply += f"\n💬 {extracted['feedback']}"
    return user_reply

## 11- Test

In [32]:
user_input = "Received salary 8000 EGP today"
llm_response = process_transaction_with_llm(user_input, user_id="027051a8-3887-4150-9cfb-a51efb9146b5", supabase=supabase) 
print(llm_response)
result = add_transaction_from_llm(llm_response, user_id="027051a8-3887-4150-9cfb-a51efb9146b5", supabase=supabase)
print(result)

 

    DATE: 2025-05-10
    INCOME: 8000
    CATEGORY: Salary
    DESCRIPTION: salary
    FEEDBACK: Congratulations on receiving your salary! Make sure to manage it wisely and plan for savings or investments.


C:\Users\Tony\anaconda3\lib\site-packages\tzlocal\utils.py:39: UserWarning: Timezone offset does not match system offset: 7200 != 10800. Please, check your config files.
  warnings.warn(msg)


✅ Added income transaction: salary, 8000.0 EGP, date: 2025-05-10, category: Salary.
💬 Congratulations on receiving your salary! Make sure to manage it wisely and plan for savings or investments.


In [38]:
user_input = "Had dinner for 150 EGP last night"
llm_response = process_transaction_with_llm(user_input, user_id="027051a8-3887-4150-9cfb-a51efb9146b5", supabase=supabase)
print(llm_response)
# Parse and save
add_transaction_from_llm(llm_response, user_id="027051a8-3887-4150-9cfb-a51efb9146b5", supabase=supabase)

 DATE: 2025-05-09
    EXPENSE: 150
    CATEGORY: Groceries
    DESCRIPTION: dinner
    FEEDBACK: Dinner sounds like a good time—hope you enjoyed it! Keep an eye on eating out expenses if you’re managing your budget.


C:\Users\Tony\anaconda3\lib\site-packages\tzlocal\utils.py:39: UserWarning: Timezone offset does not match system offset: 7200 != 10800. Please, check your config files.
  warnings.warn(msg)


'✅ Added expense transaction: dinner, 150.0 EGP, date: 2025-05-09, category: Groceries.\n💬 Dinner sounds like a good time—hope you enjoyed it! Keep an eye on eating out expenses if you’re managing your budget.'

In [35]:
user_input = "got 150 EGP"
llm_response = process_transaction_with_llm(user_input, user_id="027051a8-3887-4150-9cfb-a51efb9146b5", supabase=supabase)
print(llm_response)
# Parse and save
result = add_transaction_from_llm(llm_response, user_id="027051a8-3887-4150-9cfb-a51efb9146b5", supabase=supabase)
print(result)

2025-05-10
    DATE: 2025-05-10
    INCOME: 150
    CATEGORY: Freelance
    DESCRIPTION: payment
    FEEDBACK: Nice! It's always great to receive some extra income. Keep up the good work!


C:\Users\Tony\anaconda3\lib\site-packages\tzlocal\utils.py:39: UserWarning: Timezone offset does not match system offset: 7200 != 10800. Please, check your config files.
  warnings.warn(msg)


✅ Added income transaction: payment, 150.0 EGP, date: 2025-05-10, category: Freelance.
💬 Nice! It's always great to receive some extra income. Keep up the good work!


In [36]:
user_input = "Dinner today"
llm_response = process_transaction_with_llm(user_input, user_id="027051a8-3887-4150-9cfb-a51efb9146b5", supabase=supabase)
print(llm_response)
# Parse and save
result = add_transaction_from_llm(llm_response, user_id="027051a8-3887-4150-9cfb-a51efb9146b5", supabase=supabase)
print(result)

2025-05-10
    DATE: 2025-05-10
    EXPENSE: 
    CATEGORY: 
    DESCRIPTION: Dinner
    FEEDBACK: It seems you had dinner today. Hope it was a good experience! If you’d like to track your expenses, consider noting the amount next time.
❌ Cannot add transaction: missing information.


C:\Users\Tony\anaconda3\lib\site-packages\tzlocal\utils.py:39: UserWarning: Timezone offset does not match system offset: 7200 != 10800. Please, check your config files.
  warnings.warn(msg)


In [37]:
user_input = "50 today"
llm_response = process_transaction_with_llm(user_input, user_id="027051a8-3887-4150-9cfb-a51efb9146b5", supabase=supabase)
print(llm_response)
# Parse and save
result = add_transaction_from_llm(llm_response, user_id="027051a8-3887-4150-9cfb-a51efb9146b5", supabase=supabase)
print(result)




    DATE: 2025-05-10
    EXPENSE: 50
    CATEGORY: uncategorized
    DESCRIPTION: spending
    FEEDBACK: It looks like you spent 50 today. Consider categorizing it for better tracking next time!


C:\Users\Tony\anaconda3\lib\site-packages\tzlocal\utils.py:39: UserWarning: Timezone offset does not match system offset: 7200 != 10800. Please, check your config files.
  warnings.warn(msg)


✅ Added expense transaction: spending, 50.0 EGP, date: 2025-05-10, category: uncategorized.
💬 It looks like you spent 50 today. Consider categorizing it for better tracking next time!
